In [9]:
%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.append(os.path.join(os.getcwd(), 'src'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from slack_interactor import SlackInteractor
from claude_llm import ClaudeLLM
from db import ActionDatabase
from config import CONFIG
from agent_interface import BaseAgent
from project_manager_agent import ProjectManagerAgent
from sarcastic_agent import SarcasticAgent
from paul_graham_agent import PaulGrahamAgent  # Add this import
from drunk_agent import DrunkAgent
from runner import process_threads, execute_due_actions

In [11]:
workspaces = CONFIG['workspaces']
slack_interactors = {
    workspace_config['name']: SlackInteractor(workspace_config)
    for _, workspace_config in workspaces.items()
}

agents = {}
for _, workspace_config in workspaces.items():
    workspace_name = workspace_config['name']
    slack_interactor = slack_interactors[workspace_name]
    action_db = ActionDatabase(workspace_name)
    agents[workspace_name] = []
    
    agent_classes = {
        'ProjectManagerAgent': ProjectManagerAgent,
        'SarcasticAgent': SarcasticAgent,
        'PaulGrahamAgent': PaulGrahamAgent,
        'DrunkAgent': DrunkAgent
    }
    
    for agent_name in workspace_config.get('agents', []):
        if agent_name in agent_classes:
            agent_class = agent_classes[agent_name]
            agent = agent_class("claude", action_db, slack_interactor, workspace_name=workspace_name)
            agents[workspace_name].append(agent)
        else:
            print(f"Warning: Unknown agent type '{agent_name}' for workspace '{workspace_name}'")

In [12]:
for workspace_name, slack_interactor in slack_interactors.items():
    print(f"\nFetching new messages for workspace: {workspace_name}")
    data = slack_interactor.fetch_new_user_messages()
    threads = slack_interactor.organize_threads(data)
    print(f"Found {len(threads)} threads with new user messages in {workspace_name}.")

    if not slack_interactor.is_first_run:
        results = process_threads(agents[workspace_name], threads)
        print(f"\nChecking for due actions in {workspace_name}...")
        execute_due_actions(agents[workspace_name])
    else:
        print(f"First run for {workspace_name}. Skipping thread processing and due actions.")
        slack_interactor.is_first_run = False


Fetching new messages for workspace: agentflow
Set conversations_oldest to 1722399936.894109
Loaded 554 old messages
Found 4 new messages
Updated complete_conversations_agentflow.pkl with new messages
Found 0 threads with new user messages in agentflow.

Checking for due actions in agentflow...

Fetching new messages for workspace: legal flow
Set conversations_oldest to 1724897852.811579
Loaded 27 old messages
Found 1 new messages
Updated complete_conversations_legal flow.pkl with new messages
Rate limited. Retrying in 1.39 seconds (attempt 1/10)
Found 1 threads with new user messages in legal flow.

Processing thread in channel: agent-chat
Thread timestamp: 2024-08-29 02:20:34.838139057
Last message content:
hi
Claude claude-3-opus-20240229 prompt:

        You are an AI agent bot named Agentflow with the following characteristics:
        Personality: Professional and efficient
        Goal: Close the loop on open items, nudging and reminding people when necessary. Strive to not be 